In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf
from keras.callbacks import Callback

import copy



from tensorflow.keras.regularizers import l1, l2

from tqdm import tqdm

import numpy as np
import cv2
# import mediapipe as mp
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

# from tensorflow_model_optimization.quantization.keras import quantize_model
from collections import Counter
import random as rand
import os

In [ ]:
import numpy as np

def txt_pre_process(txt_file,label,simplify=False,simplify_level=14 ):
    label_array = []
    temp_feature_data = []
    temp_sequence_data = []
    batch_data = []

    with open(str(txt_file), 'r') as file:

        for line in file:
            values = line.strip().split('|')

            temp_feature_data = []

            for value in values:
                float_value = str(value)

                #FIRST PART OF THE SEQUENCE
                if float_value == 'START':
                    temp_sequence_data=[]

                elif float_value == 'END':
                    batch_data.append(temp_sequence_data)
                    label_array.append(label)


                elif float_value != '' and float_value != 'START':
                    if simplify:
                        float_value = round(float(value),simplify_level)
                    else:
                        float_value = float(value)
                    temp_feature_data.append(float_value)

            if temp_feature_data!=[]:
                temp_sequence_data.append(temp_feature_data)

    label_array = np.array(label_array)
    return [batch_data,label_array]

#--------------------------------------------------------------------------- paddingV1 --------------------------------------------------------------------------------
# padding can be improved probably...by using sequence
# minor issue:
# > is whether sequences had exceeded the intended number of sequences but is still right (it was performed right but slower(by an acceptable margin)) - not resolved
#    = temporary fix was just to truncate everything if it had exceeded the intended number of sequence for the sake of running it for now
#    = a reliable solution in theory could be that to randomly truncate in between the first and end sequence, in this way relevant data can be captured
def padding(pre_processed_input,optional_maxLength=0):
    padded_sequences = []
    if optional_maxLength != 0:
        max_length = optional_maxLength
    else:
        max_length = max(len(sequence) for sequence in pre_processed_input)

    for sequence in pre_processed_input:
        padding_length = max_length - len(sequence)
        if padding_length >= 0:
            padded_sequence = np.pad(sequence, ((0, padding_length), (0, 0)), mode='constant')

        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    padded_sequences = np.array(padded_sequences)

    return padded_sequences

#--------------------------------------------------------------------------- paddingV1 --------------------------------------------------------------------------------

# this is to merge correct executions and wrong executions and randomize their input and label
# positions of input and its corresponding label are the same
# introducing noise/wrong input makes the model more robust
def concatenate_randomize_batches(base_input,base_label,concat_input,concat_label):
    combined_inputs = np.concatenate((base_input,concat_input), axis = 0)
    combined_label = np.concatenate((base_label,concat_label), axis = 0)
    indices = np.random.permutation(len(combined_inputs))
    randomized_inputs = combined_inputs[indices]
    randomized_label = combined_label[indices]
    return [randomized_inputs,randomized_label]




def tally_sequence(sequence_array):
    tally_number = []
    tally_ctr = []

    for x in sequence_array:
        temp = len(x)
        if temp not in tally_number:
            tally_number.append(temp)
            tally_ctr.append(1)
        else:
            for y in range(len(tally_number)) :
                if temp == tally_number[y]:
                    tally_ctr[y] = tally_ctr[y] + 1

    tally_max = 0
    tally_number_arranged = []
    tally_ctr_arranged = []

    for x in range(len(tally_number)):
        # print(len(tally_ctr))
        tally_max = max(tally_ctr)
        for y in range(len(tally_number)):
            if tally_ctr[y] == tally_max:
                tally_number_arranged.append(tally_number[y])
                tally_ctr_arranged.append(tally_ctr[y])
                tally_ctr.pop(y)
                tally_number.pop(y)
                break

    total_ctr = 0
    for x in tally_ctr:
        total_ctr = total_ctr + x


    for x in range(len(tally_number_arranged)):
        print(tally_number_arranged[x],'-->',tally_ctr_arranged[x])


# outlier detection and removal (currently being used)
def common_length_sequence(sequences_array,threshold = 2):
    temp = []

    data = [len(seq) for seq in sequences_array]
    data_frequency = Counter(data)
    most_common_data = data_frequency.most_common()
    outlier_frequencies = [value for value, freq in data_frequency.items() if freq < threshold]
    most_common_values = [value for value, freq in most_common_data if freq >= threshold]

    print("Most Common Data Points:", most_common_values)
    print("Outlier Frequencies:", outlier_frequencies)

    for x in sequences_array:
        if len(x) in most_common_values:
            temp.append(x)
    print('-------------------applied frequency outlier detection-------------------')
    print("original num -> ", len(sequences_array))
    print("current num -> ", len(temp))
    print("removed num -> ", len(sequences_array) - len(temp))
    return temp

# outlier detection and removal (currently being used)
def apply_z_score(sequences_array,z_score_threshold = 1):
    data_points = []
    included_datapoints = []
    updated_sequences =[]

    for x in sequences_array:
        temp = len(x)
        if temp not in data_points:
            data_points.append(temp)

    data = np.array(data_points)
    mean_value = np.mean(data)
    standard_deviation = np.std(data)
    z_scores = (data - mean_value) / standard_deviation
    for x in range(len(z_scores)):
        if np.abs(z_scores[x]) <= z_score_threshold:
            included_datapoints.append(data[x])


    for x in sequences_array:
        if len(x) in included_datapoints:
            updated_sequences.append(x)
    print('-------------------applied z-score outlier detection-------------------')
    print("datapoints included -> ", included_datapoints)
    print("original num -> ", len(sequences_array))
    print("current num -> ", len(updated_sequences))
    print("removed num -> ", len(sequences_array) - len(updated_sequences))

    return updated_sequences




def paddingV2(sequences_array_input,optional_maxlength = 0):
    sequences_array = copy.deepcopy(sequences_array_input)


    output = []
    max_length = 0
    if optional_maxlength == 0:
        max_length = max(len(sequence) for sequence in sequences_array)
        expanded_max_length = int(max_length+ ((max_length) * .10))
    else:
        expanded_max_length = optional_maxlength

    # sequence = np.array(sequences_array)

    print(expanded_max_length)


    padding_length_before = 0
    padding_length_after = 0

    for seq in sequences_array:
        # print(seq)
        for x in range(expanded_max_length-len(seq)+1):
            padding_length_before = x
            padding_length_after = expanded_max_length - len(seq) - x
            padded_sequence = np.pad(seq, ((padding_length_before, padding_length_after),(0,0)), mode='constant')
            output.append(padded_sequence)

            # print(padded_sequence)
    print('------------------------applied paddingV2------------------------')
    print('max_length -> ', max_length)
    print('expanded_max_length -> ', expanded_max_length)
    print('original num set of sequences -> ', len(sequences_array))
    print('final num set of sequences -> ', len(output))

    output = np.array(output)
    return output




def convert_tf_to_tflite(tf_model,input_shape,test_dataset,name,id_number,validation_loss,validation_accuracy):
  model = tf.keras.models.load_model(tf_model)

  run_model = tf.function(lambda x: model(x))
  # This is important, let's fix the input size.
  BATCH_SIZE = input_shape[0]
  STEPS = input_shape[1]
  INPUT_SIZE = input_shape[2]
  concrete_func = run_model.get_concrete_function(
      tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

  # model directory.
  MODEL_DIR = "keras_lstm"
  model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

  converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
  tflite_model = converter.convert()


  # Run the model with TensorFlow to get expected results.
  TEST_CASES = 10

  # Run the model with TensorFlow Lite
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  for i in range(TEST_CASES):
    expected = model.predict(test_dataset[i:i+1])
    interpreter.set_tensor(input_details[0]["index"], test_dataset[i:i+1, :, :])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=5)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    interpreter.reset_all_variables()



  temp = 'converted_model_'

  temp3 = temp + str(name) + id_number + "(loss_"+ str(round(validation_loss,3)) +")" + "(acc_"+  str(round(validation_accuracy,3 )) + ")" + '.tflite'
  # Save the TFLite model to a file
  with open(temp3, "wb") as f:
    f.write(tflite_model)
  # with open("converted_model.tflite", "wb") as f:
  #     f.write(tflite_model)


# this data augmentation replaces padded index with random inputs
def populate_0_input(correct_data_input,noise_data_input):
    correct_data = copy.deepcopy(correct_data_input)
    noise_data = copy.deepcopy(noise_data_input)

    print(len(correct_data))
    index = 10
    temp = []
    temp_compilation = []
    ctr = 0
    rand_modifier =0

    for set_sequence in tqdm(correct_data, desc="populate_0_input", leave=True):
        rand_modifier = rand.randint(0,len(noise_data))

        for x in range(len(set_sequence)):
            ctr = ctr + 1
            if set_sequence[x][0] == 0:
                temp.append(noise_data[rand_modifier-1][rand.randint(0,len(noise_data[rand_modifier-1])-1)])

            else:
                temp.append(set_sequence[x])

        temp_compilation.append(temp)
        temp =[]


    return temp_compilation



class CustomEarlyStopping(Callback):
  def __init__(self, accuracy_threshold=0.95, loss_threshold=0.10):
      super(CustomEarlyStopping, self).__init__()
      self.accuracy_threshold = accuracy_threshold
      self.loss_threshold = loss_threshold

  def on_epoch_end(self, epoch, logs=None):
      if logs is None:
          logs = {}

      if logs.get('val_accuracy') is None or logs.get('val_loss') is None:
          return

      if logs.get('val_accuracy') >= self.accuracy_threshold and logs.get('val_loss') <= self.loss_threshold:
          self.model.stop_training = True
          print(f"\nTraining stopped as validation accuracy reached {logs.get('val_accuracy'):.4f} "
                f"and validation loss reached {logs.get('val_loss'):.4f}")


# this data augmentation augments data to simulate if the sequence were follow,whether there is unecessary movements in between.
# this would also reinforce the model to detect certain action that were classified as correct instead of incorrect
# this augmentation is augmenting each sequence of a list of sequence
# 1 set of sequence(example-> 1 push up):
# [seq1,seq2,seq3,seq4]
# [seq1,seq2,NOISE_SEQ34,seq4]
def data_aug_sensitivity(sequence_array_list_input,noise_sequence_list_input,num_data_aug = 3,num_aug_in_1_seq = 3,noise_seq_len = 2):
  sequence_array_list = copy.deepcopy(sequence_array_list_input)
  noise_sequence_list = copy.deepcopy(noise_sequence_list_input)

  compile = []
  temp_seq = []
  temp_storage = []
  temp_rand = []
  num = 0
  ctr1111 = 0
  temp_rand2 = 0
  temp_rand3 = 0

# per sequences
  for sequence in tqdm(sequence_array_list, desc="data_aug_seq_sensitivity", leave=True):
    # loops for the number of data augmentation per sequence
    for ctr in range(num_data_aug):
      # loops for the amount of number of augmentation in the sequence(loops to get random index)
      while len(temp_rand)!=num_aug_in_1_seq:
        num = rand.randint(0,len(sequence)-1)
        if num in temp_rand:
          continue
        else:
          temp_rand.append(num)

      #actual augmentation of the sequence
      temp_seq = sequence.copy()
      # store in a temp variable and to be edited

      # number of augmentation to be done in a sequence
      for ctr1 in range(len(temp_rand)):

        # number of sequence to be expanded(index + number of noise_seq_len)
        for ctr2 in range(noise_seq_len):
          temp_rand2 = rand.randint(0,len(noise_sequence_list)-1)
          temp_rand3 = rand.randint(0,len(noise_sequence_list[0])-1)

          if (temp_rand[ctr1] + ctr2) < len(temp_seq):
            temp_seq[temp_rand[ctr1] + ctr2] = noise_sequence_list[temp_rand2][temp_rand3]


          else:
            continue


      compile.append(temp_seq)
      temp_seq = []
      temp_rand = []


  return compile


def data_aug_seq_sensitivity(sequence_array_list_input,num_to_aug=2,num_coor_edit=3,num_sequence_edit=2):
  sequence_array_list = copy.deepcopy(sequence_array_list_input)

  compile = []
  temp = []
  rand_coor = []


  for ctr in tqdm(range(num_to_aug), desc="data_aug_coor_sensitivity", leave=True):
    for sequence in sequence_array_list:
      for ctr3 in range(num_sequence_edit):
        what_sequence = rand.randint(0,len(sequence)-1)
        for ctr2 in range(num_coor_edit):
          what_coor = rand.randint(0,len(sequence[0])-1)
          rand_coor = rand.randint(0,9999999999)
          rand_coor = rand_coor / (10 ** len(str(rand_coor)))
          sequence[what_sequence][what_coor]=rand_coor
      compile.append(sequence)
  return compile


def data_aug_coor_sensitivity(sequence_array_list_input,num_coor_edit=45,num_sequence_edit=8):
  sequence_array_list = copy.deepcopy(sequence_array_list_input)

  compile = []
  temp = []
  rand_coor = []
  temp_seq = []



  # for ctr in tqdm(range(num_to_aug), desc="data_aug_coor_sensitivity", leave=True):
  for sequence in sequence_array_list:
    temp_seq = sequence.copy()
    for ctr3 in range(num_sequence_edit):
      what_sequence = rand.randint(0,len(sequence)-1)
      num_coor_edit = rand.randint(int(num_coor_edit*.65),num_coor_edit)
      for ctr2 in range(num_coor_edit):
        what_coor = rand.randint(0,len(sequence[0])-1)
        # rand_coor = rand.randint(0,9999999999)
        rand_coor = rand.randint(0,999)
        rand_coor = rand_coor / (10 ** len(str(rand_coor)))
        temp_seq[what_sequence][what_coor]=rand_coor
    compile.append(sequence)
  return compile











In [ ]:
temp = [[[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13]],[[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13]],[[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13]],]
temp_noise = [[[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0]],[[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0]],[[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0]],]

paddingV2(temp)


5
------------------------applied paddingV2------------------------
max_length ->  5
expanded_max_length ->  5
original num set of sequences ->  3
final num set of sequences ->  6


array([[[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]],

       [[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
  

In [ ]:
temp = [[[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13]],[[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13]],[[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13],[1,2,3,4,5,6,7,8,9,10,11,12,13]],]
temp_noise = [[[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0]],[[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0]],[[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0]],]


aug = data_aug_coor_sensitivity(temp,5,5)

for x in aug:
  print(x)


[[1, 2, 3, 4, 0.8690533959, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 0.448573637, 6, 0.5771971458, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [0.6471567608, 2, 3, 4, 5, 6, 7, 0.6299563932, 0.8901594111, 0.1089798326, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]


In [ ]:
temp =[[[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]],[[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]],[[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]]]
temp_noise =[[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]
print(len(temp))
aug = data_aug_sensitivity(temp,temp_noise,2,3,2)

# aug = data_aug_coor_sensitivity(temp)
# def data_aug_sensitivity(sequence_array_list,noise_sequence_list,num_data_aug = 3,num_aug_in_1_seq = 3,noise_seq_len = 2):
for x in range(len(temp)):
  print('')
  print('original -> ',temp[x])
  print('======================')
  print('aug -> ',aug[x*2],'------------->',x*2)
  print('aug -> ',aug[x*2+1],'----------->',x*2+1)







print('qweqwe')
print(len(aug))

3


data_aug_seq_sensitivity: 100%|██████████| 3/3 [00:00<00:00, 4563.99it/s]


original ->  [[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]
aug ->  [[1, 2, 3, 4, 5], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]] -------------> 0
aug ->  [[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 2, 3, 4, 5]] -----------> 1

original ->  [[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]
aug ->  [[1, 2, 3, 4, 5], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [0, 0, 0, 0, 0]] -------------> 2
aug ->  [[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]] -----------> 3

original ->  [[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2

In [ ]:
num_to_aug=2
num_coor_edit=3
num_sequence_edit=2
print('test')
temp = [[[1,2,3,4],[1,2,3,4],[1,2,3,4]],[[1,2,3,4],[1,2,3,4],[1,2,3,4]],[[1,2,3,4],[1,2,3,4],[1,2,3,4]]]
compile = []

rand_coor = []
for ctr in range(num_to_aug):
  print('aug-----------')
  for sequence in temp:
    for ctr3 in range(num_sequence_edit):
      what_sequence = rand.randint(0,len(sequence)-1)
      for ctr2 in range(num_coor_edit):
        what_coor = rand.randint(0,len(sequence[0])-1)
        rand_coor = rand.randint(0,9999999999)
        rand_coor = rand_coor / (10 ** len(str(rand_coor)))

        sequence[what_sequence][what_coor]=rand_coor
      print(sequence)


test
aug-----------
[[1, 2, 3, 4], [1, 2, 3, 4], [0.1423094284, 2, 0.7686239527, 0.4367660063]]
[[1, 0.5017796175, 3, 0.1073473612], [1, 2, 3, 4], [0.1423094284, 2, 0.7686239527, 0.4367660063]]
[[1, 0.9731058537, 0.2545396349, 4], [1, 2, 3, 4], [1, 2, 3, 4]]
[[1, 0.9731058537, 0.2545396349, 4], [1, 2, 3, 4], [0.4022334797, 0.9948998469, 3, 0.6104141083]]
[[1, 2, 3, 4], [1, 0.8313612199, 0.9256859246, 0.640608401], [1, 2, 3, 4]]
[[0.1520550775, 2, 3, 0.7233823363], [1, 0.8313612199, 0.9256859246, 0.640608401], [1, 2, 3, 4]]
aug-----------


In [ ]:

# 0 1  -> 0
# 2 3  -> 1
# 4 5  -> 2
# 6 7  -> 3
# 8 9  -> 4
# 10 11  -> 5
# 12 13  -> 6
# 14 15  -> 7
# 16 17  -> 8
# 18 19  -> 9
# 20 21  -> 10
# 22 23  -> 11
# 24 25  -> 12
# 26 27  -> 13
# 28 29  -> 14
# 30 31  -> 15
# 32 33  -> 16
# 34 35  -> 17
# 36 37  -> 18
# 38 39  -> 19
# 40 41  -> 20
# 42 43  -> 21
# 44 45  -> 22
# 46 47  -> 23
# 48 49  -> 24
# 50 51  -> 25
# 52 53  -> 26
# 54 55  -> 27
# 56 57  -> 28
# 58 59  -> 29
# 60 61  -> 30
# 62 63  -> 31
# 64 65  -> 32

# # ----------------------------------------------------------------------------
#   left_upper_arm_sequence = []
#   left_lower_arm_sequence = []
#   left_hand_sequence = []

#   right_upper_arm_sequence = []
#   right_lower_arm_sequence = []
#   right_hand_sequence = []

#   left_upper_leg_sequence = []
#   left_lower_leg_sequence = []
#   left_feet_sequence = []

#   right_upper_leg_sequence = []
#   right_lower_leg_sequence = []
#   right_feet_sequence = []

#   body_sequence = []
#   head_sequence = []

#   # ------------------------------------------------------------------------------
#   left_upper_arm_sequence_noise = []
#   left_lower_arm_sequence_noise = []
#   left_hand_sequence_noise = []

#   right_upper_arm_sequence_noise = []
#   right_lower_arm_sequence_noise = []
#   right_hand_sequence_noise = []

#   left_upper_leg_sequence_noise = []
#   left_lower_leg_sequence_noise = []
#   left_feet_sequence_noise = []

#   right_upper_leg_sequence_noise = []
#   right_lower_leg_sequence_noise = []
#   right_feet_sequence_noise = []

#   body_sequence_noise = []
#   head_sequence_noise = []




reinforce your noise/incorrect dataset by having correct data but with randomized index replaced with random sequence

in theory this would make the model more sensitive to changes

we may also add values unto the model to simulate certain movement going more than what is intended

In [ ]:
# this is as a whole exercise
def streamlined_process(correct_execution,noise_data):
  id_num = str(rand.randint(1000,9999))
  base_data = txt_pre_process(correct_execution,1,False,4)
  base_data_noise = txt_pre_process(noise_data,0,False,4)

  loop = 0

  temp_correct_start=[]
  temp_correct_wrong=[]

  temp_wrong_store_train = []

  data = base_data[0]
  data_noise = base_data_noise[0]

  best_val_loss = float('inf')
  best_val_accuracy = 0.0
  best_model = None



  print('')
  print('----------------------------correct data augmentation ----------------------------')
  aug = common_length_sequence(data)
  aug2 = apply_z_score(aug,1)
  aug3 = paddingV2(aug2)
  aug4 = populate_0_input(aug3,data_noise)
  aug4 = np.array(aug4)
  aug4 = aug4.reshape(-1,len(aug3[0]),len(aug3[0][0]))
  combined_inputs = np.concatenate((aug4,aug3), axis = 0)
  combined_inputs = aug4
  print('concat -> ', len(combined_inputs))


  print('')
  print('----------------------------data noise data augmentation ----------------------------')
  aug_noise_data1 = paddingV2(data_noise,len(aug3[0]))
  aug_noise_data2 = populate_0_input(aug_noise_data1,data_noise)

# testing...
  # data_aug_coor_sensitivity()

# original
  # aug_noise_data3 = aug_noise_data2[0:len(combined_inputs)]

# testing(temporary)
  aug_noise_data3 = aug_noise_data2[:]

  aug_noise_data4 = np.array(aug_noise_data3)

  aug_noise_data5 = aug_noise_data4.reshape(-1,len(aug_noise_data4[0]),len(aug_noise_data4[0][0]))

# testing(temporary)
  aug_noise_data7 = data_aug_coor_sensitivity(aug_noise_data5,num_sequence_edit=int(len(aug_noise_data5[0])*.70))

  # aug_noise_data6 = data_aug_sensitivity(aug4,aug_noise_data5,1,2,2)
  aug_noise_data6 = data_aug_sensitivity(aug4,aug_noise_data5,1,2,int(len(combined_inputs)*.35))

# original
  # aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5), axis = 0)

# testing(temporary)
  aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5,aug_noise_data7), axis = 0)


  print('aug_noise_data5--->',aug_noise_data5.shape)


  loop = int(len(aug_noise_data5)/len(combined_inputs))

  model = Sequential()
  model.add(LSTM(len(aug4[0]), return_sequences=True, activation='relu',  input_shape=(len(aug4[0]), len(aug4[0][0]))))
  model.add(LSTM(len(aug4[0]) + int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=True,  activation='relu'))
  model.add(Bidirectional(LSTM(len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=True, dropout=0.3, recurrent_dropout=0.3, activation='relu')))
  model.add(LSTM(len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=False, activation='relu'))
  # model.add(BatchNormalization())
  model.add(Dense(len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .3)), activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  custom_early_stopping = CustomEarlyStopping(accuracy_threshold=0.97, loss_threshold=0.05)
  model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy'])




  for x in range(loop):
    temp_wrong_store_train = aug_noise_data5[x*len(combined_inputs):len(combined_inputs)*(x+1)]

    aug3_label = np.ones(len(combined_inputs))
    aug_noise_label = np.zeros(len(temp_wrong_store_train))

    rand_batches=concatenate_randomize_batches(combined_inputs,aug3_label,temp_wrong_store_train,aug_noise_label)

    X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.3, random_state=42)

    # history=model.fit(X_train, y_train, epochs=200, batch_size =256  , validation_data=(X_test, y_test), callbacks=[custom_early_stopping])
    history=model.fit(X_train, y_train, epochs=200, batch_size =256  , validation_data=(X_test, y_test), callbacks=[custom_early_stopping])


    val_loss = min(history.history['val_loss'])
    val_accuracy = max(history.history['val_accuracy'])
    temp_wrong_store_train=[]


    if val_loss < best_val_loss and val_accuracy > best_val_accuracy:
      best_model = model.get_weights()
      best_val_loss = val_loss
      best_val_accuracy = val_accuracy

    # testing(temporary)
    model.set_weights(best_model)

  X_train = X_train.astype(np.float32)
  X_test = X_test.astype(np.float32)
  model.save('testingModel')
  convert_tf_to_tflite('/content/testingModel',[1,len(aug3[0]),len(aug3[0][0])], X_test,'whole_model',id_num,val_loss,val_accuracy)



















# uncomment this to run training model as a whole(this means that this model can determine the motion as a whole but not specify what part if correct or not)
streamlined_process('/content/drive/MyDrive/Colab Notebooks/correct_new_2.txt','/content/drive/MyDrive/Colab Notebooks/wrong_new_2.txt')





----------------------------correct data augmentation ----------------------------
Most Common Data Points: [17, 16, 19, 18, 20, 21, 22, 12, 15, 9, 14, 10, 6]
Outlier Frequencies: [5, 26, 27, 37]
-------------------applied frequency outlier detection-------------------
original num ->  102
current num ->  98
removed num ->  4
-------------------applied z-score outlier detection-------------------
datapoints included ->  [19, 17, 16, 14, 18, 20, 12, 15]
original num ->  98
current num ->  82
removed num ->  16
22
------------------------applied paddingV2------------------------
max_length ->  20
expanded_max_length ->  22
original num set of sequences ->  82
final num set of sequences ->  470
470


populate_0_input: 100%|██████████| 470/470 [00:00<00:00, 40409.21it/s]


concat ->  470

----------------------------data noise data augmentation ----------------------------
22
------------------------applied paddingV2------------------------
max_length ->  0
expanded_max_length ->  22
original num set of sequences ->  160
final num set of sequences ->  1550
1550


data_aug_seq_sensitivity: 100%|██████████| 470/470 [00:00<00:00, 1355.08it/s]


aug_noise_data5---> (3570, 22, 66)


Epoch 1/200
3/3 [==============================] - 52s 716ms/step - loss: 0.6945 - accuracy: 0.4894 - val_loss: 0.6922 - val_accuracy: 0.5248
Epoch 2/200
3/3 [==============================] - 1s 305ms/step - loss: 0.6931 - accuracy: 0.4894 - val_loss: 0.6917 - val_accuracy: 0.5248
Epoch 3/200
3/3 [==============================] - 1s 316ms/step - loss: 0.6923 - accuracy: 0.4894 - val_loss: 0.6911 - val_accuracy: 0.5248
Epoch 4/200
3/3 [==============================] - 1s 313ms/step - loss: 0.6921 - accuracy: 0.4894 - val_loss: 0.6903 - val_accuracy: 0.5248
Epoch 5/200
3/3 [==============================] - 1s 296ms/step - loss: 0.6913 - accuracy: 0.4894 - val_loss: 0.6890 - val_accuracy: 0.5248
Epoch 6/200
3/3 [==============================] - 2s 749ms/step - loss: 0.6901 - accuracy: 0.5137 - val_loss: 0.6870 - val_accuracy: 0.5248
Epoch 7/200
3/3 [==============================] - 2s 690ms/step - loss: 0.6889 - accuracy: 0.4924 - val_loss: 0.6834 - val_accuracy: 0.5248
Epoch 8/200


1/1 [==============================] - 1s 710ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 36ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 35ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 39ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 38ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 35ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 42ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 36ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==================

In [ ]:
# this is for the individual models
# this is experimental
def streamlined_process(correct_execution,noise_data,loop=5):
  id_num = str(rand.randint(1000,9999))
  base_data = txt_pre_process(correct_execution,1,False,4)
  base_data_noise = txt_pre_process(noise_data,0,False,4)

  temp_correct_start=[]
  temp_correct_wrong=[]

  data = base_data[0]
  data_noise = base_data_noise[0]



  print('')
  print('----------------------------correct data augmentation ----------------------------')
  aug = common_length_sequence(data)
  aug2 = apply_z_score(aug,1)
  aug3 = paddingV2(aug2)
  aug4 = populate_0_input(aug3,data_noise)
  aug4 = np.array(aug4)
  aug4 = aug4.reshape(-1,len(aug3[0]),len(aug3[0][0]))
  combined_inputs = np.concatenate((aug4,aug3), axis = 0)
  combined_inputs= aug4
  print('concat -> ', len(combined_inputs))



  print('')
  print('----------------------------data noise data augmentation ----------------------------')
  aug_noise_data1 = paddingV2(data_noise,len(aug3[0]))
  aug_noise_data2 = populate_0_input(aug_noise_data1,data_noise)
  aug_noise_data3 = aug_noise_data2[0:len(combined_inputs)]
  aug_noise_data4 = np.array(aug_noise_data3)
  aug_noise_data5 = aug_noise_data4.reshape(-1,len(aug_noise_data4[0]),len(aug_noise_data4[0][0]))

  aug_noise_data6 = data_aug_sensitivity(aug4,aug_noise_data5,1,1,int(len(combined_inputs)*.35))
  aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5), axis = 0)
  # aug_noise_data6 = data_aug_coor_sensitivity(aug4)
  # aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5), axis = 0)


  correct_data_set = []
  noise_data_set = []
  rand_batches = []

  print('testtesttesttest=======>',len(aug_noise_data5))


  for x in range(14):
    correct_data_set.append([])
    noise_data_set.append([])
    rand_batches.append([])


  for exercise in combined_inputs:

    left_upper_arm = []
    left_lower_arm = []
    left_hand = []
    right_upper_arm = []
    right_lower_arm = []
    right_hand = []
    left_upper_leg = []
    left_lower_leg = []
    left_feet = []
    right_upper_leg = []
    right_lower_leg = []
    right_feet = []
    head =[]
    body = []



#------------------------ generating the correct input of certain part------------------------------
    for sequence in exercise:
      # 11,13
      left_upper_arm.append([sequence[22],sequence[23],sequence[26],sequence[27]])
      # 13,15
      left_lower_arm.append([sequence[26],sequence[27],sequence[30],sequence[31]])
      # 15,17,19,21
      left_hand.append([sequence[30],sequence[31],sequence[34],sequence[35],sequence[38],sequence[39],sequence[42],sequence[43]])

      # 12,14
      right_upper_arm.append([sequence[24],sequence[25],sequence[28],sequence[29]])
      # 14,16
      right_lower_arm.append([sequence[28],sequence[29],sequence[32],sequence[33]])
      # 16,18,20,22
      right_hand.append([sequence[32],sequence[33],sequence[36],sequence[37],sequence[40],sequence[41],sequence[44],sequence[45]])

      # 23,25
      left_upper_leg.append([sequence[46],sequence[47],sequence[50],sequence[51]])
      # 25,27
      left_lower_leg.append([sequence[50],sequence[51],sequence[54],sequence[55]])
      # 27,29,31
      left_feet.append([sequence[54],sequence[55],sequence[58],sequence[59],sequence[62],sequence[63]])

      # 24,26
      right_upper_leg.append([sequence[48],sequence[49],sequence[52],sequence[53]])
      # 26,28
      right_lower_leg.append([sequence[52],sequence[53],sequence[56],sequence[57]])
      # 28,30,32
      right_feet.append([sequence[56],sequence[57],sequence[60],sequence[61],sequence[64],sequence[65]])

      # 11,12,23,24
      body.append([sequence[22],sequence[23],sequence[24],sequence[25],sequence[46],sequence[47],sequence[48],sequence[49]])
      # 7,8,9,10
      head.append([sequence[14],sequence[15],sequence[16],sequence[17],sequence[18],sequence[19],sequence[20],sequence[21]])


    correct_data_set[0].append(left_upper_arm)
    correct_data_set[1].append(left_lower_arm)
    correct_data_set[2].append(left_hand)

    correct_data_set[3].append(right_upper_arm)
    correct_data_set[4].append(right_lower_arm)
    correct_data_set[5].append(right_hand)

    correct_data_set[6].append(left_upper_leg)
    correct_data_set[7].append(left_lower_leg)
    correct_data_set[8].append(left_feet)

    correct_data_set[9].append(right_upper_leg)
    correct_data_set[10].append(right_lower_leg)
    correct_data_set[11].append(right_feet)

    correct_data_set[12].append(body)
    correct_data_set[13].append(head)




  for x in range(len(correct_data_set)):
    correct_data_set[x] = np.array(correct_data_set[x])

  # for x in range(len(noise_data_set)):
  #   noise_data_set[x] = np.array(noise_data_set[x])

  print('len of correct data set ---->',len(correct_data_set[0]))
  print('len of noise data set ---->',len(noise_data_set[0]))

  correct_data_set_label = np.ones(len(combined_inputs))
  noise_data_set_label = np.zeros(len(aug_noise_data5))

  # for x in range(len(correct_data_set)):
  #   rand_batches[x]=concatenate_randomize_batches(correct_data_set[x],correct_data_set_label,noise_data_set[x],noise_data_set_label)



  loop = int(len(aug_noise_data5)/len(combined_inputs))


  data_set_name=['left_upper_arm','left_lower_arm','left_hand','right_upper_arm','right_lower_arm','right_hand','left_upper_leg','left_lower_leg','left_feet','right_upper_leg','right_lower_leg','right_feet','head','body']
  for x in range(len(data_set_name)-1):
    print('progress -> ',x,'/',len(data_set_name)-1)



    best_val_loss = float('inf')
    best_val_accuracy = 0.0
    best_model = None

    for y in range(loop):

      noise_data_set=[]


    #------------------------------ generating noise ---------------------------------------
      for exercise in aug_noise_data5[len(correct_data_set[x])*y:len(correct_data_set[x]) + (len(correct_data_set[x])*y)]:
      # for exercise in aug_noise_data5[0:450]:
        # print('batchin noise ->',len(correct_data_set[x])*y,'<==--==>',len(correct_data_set[x]) + (len(correct_data_set[x])*y))


        left_upper_arm = []
        left_lower_arm = []
        left_hand = []
        right_upper_arm = []
        right_lower_arm = []
        right_hand = []
        left_upper_leg = []
        left_lower_leg = []
        left_feet = []
        right_upper_leg = []
        right_lower_leg = []
        right_feet = []
        head =[]
        body = []



        for sequence in exercise:
          # 11,13
          # print(sequence[22])
          if x == 0:
            left_upper_arm.append([sequence[22],sequence[23],sequence[26],sequence[27]])
          # 13,15
          if x == 1:
            left_lower_arm.append([sequence[26],sequence[27],sequence[30],sequence[31]])
          # 15,17,19,21
          if x == 2:
            left_hand.append([sequence[30],sequence[31],sequence[34],sequence[35],sequence[38],sequence[39],sequence[42],sequence[43]])

          # 12,14
          if x == 3:
            right_upper_arm.append([sequence[24],sequence[25],sequence[28],sequence[29]])
          # 14,16
          if x == 4:
            right_lower_arm.append([sequence[28],sequence[29],sequence[32],sequence[33]])
          # 16,18,20,22
          if x == 5:
            right_hand.append([sequence[32],sequence[33],sequence[36],sequence[37],sequence[40],sequence[41],sequence[44],sequence[45]])

          # 23,25
          if x == 6:
            left_upper_leg.append([sequence[46],sequence[47],sequence[50],sequence[51]])
          # 25,27
          if x == 7:
            left_lower_leg.append([sequence[50],sequence[51],sequence[54],sequence[55]])
          # 27,29,31
          if x == 8:
            left_feet.append([sequence[54],sequence[55],sequence[58],sequence[59],sequence[62],sequence[63]])

          # 24,26
          if x == 9:
            right_upper_leg.append([sequence[48],sequence[49],sequence[52],sequence[53]])
          # 26,28
          if x == 10:
            right_lower_leg.append([sequence[52],sequence[53],sequence[56],sequence[57]])
          # 28,30,32
          if x == 11:
            right_feet.append([sequence[56],sequence[57],sequence[60],sequence[61],sequence[64],sequence[65]])

          # 11,12,23,24
          if x == 12:
            body.append([sequence[22],sequence[23],sequence[24],sequence[25],sequence[46],sequence[47],sequence[48],sequence[49]])
          # 7,8,9,10
          if x == 13:
            head.append([sequence[14],sequence[15],sequence[16],sequence[17],sequence[18],sequence[19],sequence[20],sequence[21]])

        if x == 0:
          noise_data_set.append(left_upper_arm)
        if x == 1:
          noise_data_set.append(left_lower_arm)
        if x == 2:
          noise_data_set.append(left_hand)

        if x == 3:
          noise_data_set.append(right_upper_arm)
        if x == 4:
          noise_data_set.append(right_lower_arm)
        if x == 5:
          noise_data_set.append(right_hand)

        if x == 6:
          noise_data_set.append(left_upper_leg)
        if x == 7:
          noise_data_set.append(left_lower_leg)
        if x == 8:
          noise_data_set.append(left_feet)


        if x == 9:
          noise_data_set.append(right_upper_leg)
        if x == 10:
          noise_data_set.append(right_lower_leg)
        if x == 11:
          noise_data_set.append(right_feet)


        if x == 12:
          noise_data_set.append(body)
        if x == 13:
          noise_data_set.append(head)

# ===================================================================================================================================================

      print('correct->',len(correct_data_set[x]),'  incorrect->',len(noise_data_set[x]))

      rand_batches=concatenate_randomize_batches(correct_data_set[x],correct_data_set_label,noise_data_set,noise_data_set_label)





      print('loop -> ',y,'/',loop)
      X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.2, random_state=42)

      # history = model.fit(X_train[y*len(rand_batches[x][0]):len(rand_batches[x][0])*(y+1)], y_train[y*len(rand_batches[x][1]):len(rand_batches[x][1])*(y+1)], epochs=200, batch_size =128 , validation_data=(X_test[y*len(rand_batches[x][0]):len(rand_batches[x][0])*(y+1)], y_test[y*len(rand_batches[x][1]):len(rand_batches[x][1])*(y+1)]), callbacks=[custom_early_stopping])

      if y == 0:
        model_base_modifier = 10
        model = Sequential()
        model.add(LSTM(model_base_modifier+len(aug4[0]), return_sequences=True, activation='relu',  input_shape=(len(correct_data_set[x][0]), len(correct_data_set[x][0][0]))))
        model.add(LSTM(model_base_modifier+len(aug4[0]) + int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=True,  activation='relu' ))
        model.add(Bidirectional(LSTM(model_base_modifier+len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .5)), return_sequences=True, dropout=0.4, recurrent_dropout=0.4, activation='relu')))
        model.add(LSTM(model_base_modifier+len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=False,  activation='relu'))
        # model.add(BatchNormalization())
        model.add(Dense(model_base_modifier+len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), activation='relu'))

        model.add(Dense(1,activation='sigmoid'))

        custom_early_stopping = CustomEarlyStopping(accuracy_threshold=0.97, loss_threshold=0.05)
        model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy'])



      history = model.fit(X_train, y_train, epochs=150, batch_size =128 , validation_data=(X_test, y_test), callbacks=[custom_early_stopping])

      val_loss = min(history.history['val_loss'])
      val_accuracy = max(history.history['val_accuracy'])

      if val_loss < best_val_loss and val_accuracy > best_val_accuracy:
        best_model = model.get_weights()
        best_val_loss = val_loss
        best_val_accuracy = val_accuracy

      model.set_weights(best_model)



    model.save('testingModel')


    # X_train = X_train[x].astype(np.float32)
    # X_test = X_test[x].astype(np.float32)

    # convert_tf_to_tflite('/content/testingModel',[1,len(X_train[0]),len(X_train[0][0])], X_test,data_set_name[x],id_num,val_loss,val_accuracy)
    # convert_tf_to_tflite('/content/testingModel',[1,len(correct_data_set[x][0]),len(correct_data_set[x][0][0])], X_test,data_set_name[x],id_num,val_loss,val_accuracy)


    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    model.save('testingModel')

    print('param 1 ->',len(noise_data_set[0]) )
    print('param 1 ->',len(noise_data_set[0][0]))
    print('current x ->',x)
    convert_tf_to_tflite('/content/testingModel',[1,len(noise_data_set[0]),len(noise_data_set[0][0])], X_test,data_set_name[x],id_num,best_val_loss,best_val_accuracy)




    clear_output(wait=True)





# uncomment this to run the individual landmarks training training(this means that it can determine individual parts of the body if spefic parts are incorrect or correct)
# streamlined_process('/content/drive/MyDrive/Colab Notebooks/correct_new_2.txt','/content/drive/MyDrive/Colab Notebooks/wrong_new_2.txt')
3

3

analysis(sept 9 2023):

paddingV2 function possibly causing problems

made a function that augments individual coordinates can make or break, still needds more testing

# New section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
